In [89]:
import os
import pandas as pd
import numpy as np
import h5py
from collections import Counter

import pandas as pd

def read_info_from_predixcan(fn, id2gene = None):
    ## inputs:
    ## fn: path to the file outputed from the S-Predixcan
    ## id2gene: an list or array of ensembl ids to order the output vector if provided
    ##          otherwise the output array is arranged as it is in the S-Predixcan file
    
    df_meta = pd.read_csv(fn)
    df_meta = df_meta.dropna(axis = 0)
    if id2gene == None:
        id2gene = list(df_meta.index)
    
    gene2zscore = {item.gene:item.zscore for item in df_meta.itertuples()}
    gene2var = {item.gene:item.VAR_g for item in df_meta.itertuples()}

    var_g = np.zeros(len(id2gene))
    for i, g in enumerate(id2gene):
        if g in gene2var:
            var_g[i] = gene2var[g]

    z_g = np.zeros(len(id2gene))
    for i, g in enumerate(id2gene):
        if g in gene2zscore:
            z_g[i] = gene2zscore[g]
    
    return z_g, var_g
                
def compute_gwas_z(pca_genes, pca_weight, pca_sig, gene2zscore, gene2var):
    ## inputs:
    ## df_spca: a (lantent factor, gene) numpy array of the lantent factor x gene matrix
    ## sig_gtex: a (gene, ) numpy array of the per-gene variance from GTEX
    ## df_cov: a (gene, gene) numpy array of the gene-gene covariance
    ## z_g: a (gene, ) numpy array of per-gene GWAS associations from s-predixscan
    ## var_g: a (gene, ) numpy array of per-gene GWAS variance from s-predixscan
    ## is_deconvolute: a bool variable. If True the the deconvolution is performed.
    
    
    z_lantent = 0.0
    for i, g in enumerate(pca_genes):
        if g in gene2zscore and g in gene2var:
            w = pca_weight[i]
            z = gene2zscore[g]
            s = np.sqrt(gene2var[g])
            
            z_lantent += w*z*s
    z_lantent = z_lantent / pca_sig
    #print c, c / np.sqrt(pca_weight.dot(df_1k[pca_genes].cov().values).dot(pca_weight))
    '''
    z_g = np.zeros(*pca_weight.shape)
    sig_g = np.zeros(*pca_weight.shape)
    for i, g in enumerate(pca_genes):
        if g in gene2zscore and g in gene2var:
            z_g[i] = gene2zscore[g]
            sig_g[i] = np.sqrt(gene2var[g])
            
    z_lantent = np.sum(pca_weight * (z_g * sig_g)) / pca_sig
    '''
    return z_lantent

In [2]:
fns = sorted(os.listdir('../processed_data/adj_expression/'))

In [3]:
tissue = fns[0].split('-')[0]

In [8]:
df_1k = pd.read_csv('../processed_data/predixcan_1kgenome/TW_Adipose_Subcutaneous_0.5.db_predicted_expression.txt.gz', sep = '\t')

In [11]:
df_1k.columns = [g.split('.')[0] for g in df_1k.columns]

In [12]:
with h5py.File('../processed_data/model_weights/%s-eigengene_model.h5' % tissue, 'r') as reader:
    print reader.keys()
    print reader['weights'].keys()

[u'gene_clusters', u'model_genes', u'weights']
[u'cluster0005', u'cluster0007', u'cluster0015', u'cluster0031', u'cluster0032', u'cluster0050', u'cluster0051', u'cluster0056', u'cluster0060', u'cluster0068', u'cluster0072', u'cluster0097', u'cluster0126', u'cluster0127', u'cluster0128', u'cluster0131', u'cluster0153', u'cluster0156', u'cluster0166', u'cluster0171', u'cluster0184', u'cluster0193', u'cluster0226', u'cluster0240', u'cluster0249', u'cluster0258', u'cluster0268', u'cluster0299', u'cluster0323', u'cluster0352', u'cluster0372', u'cluster0403', u'cluster0405', u'cluster0408', u'cluster0414', u'cluster0421', u'cluster0429', u'cluster0430', u'cluster0434', u'cluster0470', u'cluster0475', u'cluster0478', u'cluster0479', u'cluster0494', u'cluster0498', u'cluster0515', u'cluster0519', u'cluster0526', u'cluster0541', u'cluster0555', u'cluster0571', u'cluster0600', u'cluster0601', u'cluster0647', u'cluster0675', u'cluster0698', u'cluster0725', u'cluster0735', u'cluster0736', u'cluste

In [14]:
#!/usr/bin/env python
import pandas
import sqlite3

connection = sqlite3.connect("../data/gwas_g2p/metaxcan_results_v1.5.db")

query = 'SELECT g.gene_name, m.zscore, m.n_snps_used, m.n_snps_model, p.tag as phenotype, t.tissue as tissue, g.gene ' 
query += ' FROM gene AS g INNER JOIN metaxcan_result AS m ON g.id = m.gene_id' 
query += ' INNER JOIN tissue AS t ON t.id = m.tissue_id  INNER JOIN pheno AS p ON p.id = m.pheno_id'

pi = pandas.read_sql_query(query, connection)
pi = pi[pi.tissue != 'DGN_WB']
#res = {}
#for key, item in pi.groupby('phenotype'):
#    res[key] = [(un, um) for un, um in zip(item.n_snps_used, item.n_snps_model)]

In [83]:
df_sig = pd.read_csv('../processed_data/GTEX_info/sig_%s.csv' % tissue, header = None)
gene2var = {g:z for g, z in zip(df_sig[0], df_sig[1])}


In [15]:
pi = pi[(pi.phenotype == 'CARDIoGRAM_C4D_CAD_ADDITIVE')]
pi = pi[(pi.tissue == tissue)]


In [16]:
gene2z = {g:z for g, z in zip(pi.gene, pi.zscore)}

In [49]:
pca_models = []
with h5py.File('../processed_data/model_weights/%s-eigengene_model.h5' % tissue, 'r') as reader:
    for k in sorted(reader['weights'].keys()):
        pca_genes = reader['weights'][k]['genes'][...]
        pca_weight = reader['weights'][k]['pca_weights'][...]
        pca_sig = reader['weights'][k]['pca_sig'][...]
        pca_models.append((k, pca_genes, pca_weight, pca_sig))

In [42]:
import pickle
with open('ensembl2genename.p', 'rb') as reader:
    gid2gn = pickle.load(reader)

In [84]:
res = []
for k, pca_genes, pca_weight, pca_sig in pca_models:
    
    sig_1k = (df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()
    
    res.append((sig_1k/pca_sig))

In [85]:
np.sqrt(pca_weight.dot(df_1k[pca_genes].cov().values).dot(pca_weight))

0.22048267624575663

In [86]:
(df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()

0.22043864575580271

In [90]:
res = []
for k, pca_genes, pca_weight, pca_sig in pca_models:
    
    sig_1k = (df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()
    
    z = compute_gwas_z(pca_genes, pca_weight, sig_1k, gene2z, gene2var)
    res.append(z)
    if abs(z) > 10.:
        print k, z, sig_1k
        for g in pca_genes:
            if g in gid2gn:
                print gid2gn[g], gene2z[g]
        print

cluster0166 11.515118080974506 0.22643263440882389
RCN1 -1.00214555438
GAMT -1.37714670559
RGS19 -1.61665403209
OPRL1 -2.12711650179
ACOX2 -0.907701430859

cluster0352 14.523417370111686 0.16588229726614628
CYP2C8 2.85090860373
KIAA1143 1.7851834871
KIAA1109 0.819427790664
PAM 0.661283355936
COPS6 0.858632237123
TFR2 1.07835116205

cluster0403 -15.853697371476576 0.13987221780534614
DNASE2B -0.834277033806
LIPA 4.38040796123
NRIP3 0.773883407101
HAPLN4 -0.343509686109
NCEH1 -0.0693213341766
COL22A1 1.45981962557

cluster0919 -11.67646382147323 0.18669706275022313
PPIF -0.201147034764
HP -3.54242018103
HPR -0.426820981048
TNFSF14 -0.24059784151
C3 0.373959181055
BOC 1.20137661603

cluster0957 10.141037560061076 0.24550890289866795
PTGER3 0.433579489673
GSDMB -1.93917572384
ORMDL3 -0.496700647051
ACVR1C 0.131751056586
ADH1A -2.16440544191
ADH1B -2.26343967576
GHR -0.128760142032
MEST -1.24064750115
EPB41L4B 0.200493828405

cluster1084 -10.56326961918639 0.1632225350860384
SLC35E4 0.06570

In [91]:
sorted(res)

[-15.853697371476576,
 -13.48493230924508,
 -12.070447871792327,
 -11.67646382147323,
 -10.56326961918639,
 -10.49329198409526,
 -9.868241704903713,
 -7.418976561635043,
 -7.292203937508301,
 -7.186387031670681,
 -7.092451035911698,
 -6.902526259448165,
 -6.842547919278909,
 -6.370069176327468,
 -6.204478039882395,
 -6.055079072323998,
 -5.982646874214061,
 -5.9010247029488045,
 -5.730032286949164,
 -5.690087997300045,
 -5.557543356539627,
 -5.557091132261948,
 -5.261006505220682,
 -5.1780829281137075,
 -5.055971289646944,
 -4.872634649794114,
 -4.644200763672877,
 -4.6273103149507975,
 -4.386753371235492,
 -4.355273518826337,
 -4.312476828947687,
 -4.267693295680719,
 -4.033600870025845,
 -3.987521124192526,
 -3.8813396863781877,
 -3.8516042169604763,
 -3.8117459103422444,
 -3.7676431585591663,
 -3.7436602650314423,
 -3.487669175602675,
 -3.3220078723788173,
 -3.2610033511861394,
 -3.2194891859756534,
 -3.0187750452610165,
 -2.995684466247848,
 -2.915438466302686,
 -2.831826769991312,

In [22]:
g0 = 'ENSG00000144426'

with h5py.File('../processed_data/model_weights/%s-eigengene_model.h5' % tissue, 'r') as reader:
    id2genes = reader['model_genes'][...]

In [23]:
g0 in id2genes

True

In [115]:
g0 = 'ENSG00000163596'
model_genes = []

with h5py.File('../processed_data/model_weights/%s-eigengene_model.h5' % tissue, 'r') as reader:
    
    for k in sorted(reader['weights'].keys()):
        pca_genes = reader['weights'][k]['genes'][...]
        pca_weight = reader['weights'][k]['pca_weights'][...]
        pca_sig = reader['weights'][k]['pca_sig'][...]
        
        model_genes += list(pca_genes)
        if g0 in pca_genes:
            print k
            for g in pca_genes:
                if g in gene2z:
                    print gene2z[g]
                else:
                    print g
            break

cluster0541
1.48760657373
-0.0367478480274
-1.68049932579
-0.0121706918046
0.529826229504
8.59497582584
-0.366676115344
1.16410866609
0.42572162606
0.871975521237


In [128]:
res = []
for k, pca_genes, pca_weight, pca_sig in pca_models:
    if k != 'cluster0541':
        continue
        
    sig_1k = (df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()
    
    z = compute_gwas_z(pca_genes, pca_weight, sig_1k, gene2z, gene2var)
    res.append(z)

    print k, z, sig_1k
    c = 0.
    for i, g in enumerate(pca_genes):
        if g in gid2gn:
            w, z ,s = pca_weight[i], gene2z[g], gene2var[g]
            print w, z, w*z* df_1k[g].std()
            c += w*z* df_1k[g].std()  
    print c, c / sig_1k

    break

cluster0541 -7.092451035911698 0.2505344025025094
-0.3279051230852034 1.48760657373 -0.07561866242574485
-0.4474640392831214 -0.0367478480274 0.00010832081929261267
-0.4740599353578534 -1.68049932579 0.09351564551628003
-0.37433407631592835 -0.0121706918046 0.0006157477105800467
-0.2653495600750051 0.529826229504 -0.01106950963449224
-0.22517683146453268 8.59497582584 -0.18625104030498407
-0.23900471683960836 -0.366676115344 0.04038944553541047
-0.21250515710304704 1.16410866609 -0.14189159855146186
-0.20661771494209857 0.42572162606 -0.04056444244321598
-0.24760405841755967 0.871975521237 -0.037889323016540695
-0.35865541679487656 -1.4315615468868959


In [136]:
(df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).mean()

0.10581670645483005

In [123]:
for g in pca_genes:
    if g in gid2gn:
        print gid2gn[g], gene2z[g]


WDR63 1.48760657373
HOXB5 -0.0367478480274
HOXB6 -1.68049932579
HOXB7 -0.0121706918046
TMEM87B 0.529826229504
ICA1L 8.59497582584
ATG10 -0.366676115344
ATP6AP1L 1.16410866609
SLC26A8 0.42572162606
RGS22 0.871975521237


In [118]:
np.sqrt(pca_weight.dot(df_1k[pca_genes].cov().values).dot(pca_weight))

0.250584444329135

In [119]:
c = 0.0
for w, z, s in zip(pca_weight, z_g, sig_g):
    print w, z ,s, w*z*s
    c += w*z*s
print c, c / np.sqrt(pca_weight.dot(df_1k[pca_genes].cov().values).dot(pca_weight))

-0.3279051230852034 1.48760657373 1.198743435306321 -0.5847396355057406
-0.4474640392831214 -0.0367478480274 1.0313796614826183 0.01695932697225355
-0.4740599353578534 -1.68049932579 1.020389510902577 0.8129008565315835
-0.37433407631592835 -0.0121706918046 0.9862827841025108 0.004493410346768158
-0.2653495600750051 0.529826229504 0.6071156212213618 -0.08535387333748941
-0.22517683146453268 8.59497582584 0.768021602440322 -1.486420885980774
-0.23900471683960836 -0.366676115344 0.582685832087069 0.051065025378479276
-0.21250515710304704 1.16410866609 0.7844033880236623 -0.1940450002226359
-0.20661771494209857 0.42572162605999997 1.1475510742664137 -0.10094046251640289
-0.24760405841755967 0.871975521237 0.976457510221515 -0.21082174422648273
-1.7769029825604412 -7.091034670238881


In [26]:
z_g = np.zeros(*pca_weight.shape)
sig_g = np.zeros(*pca_weight.shape)

for i, g in enumerate(pca_genes):
    if g in gene2z and g in gene2var:
        z_g[i] = gene2z[g]
        sig_g[i] = np.sqrt(gene2var[g])
            
z_lantent = np.sum(pca_weight * (z_g * sig_g)) / pca_sig

In [37]:
sig_1k = (df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()

In [39]:
np.sum(pca_weight * (z_g * sig_g)) / sig_1k

-7.092451035911698

In [38]:
sig_1k

0.2505344025025094